# 🎨 Agentic Design Patterns with GitHub Models (Python)

## 📋 Learning Objectives

This notebook demonstrates essential design patterns for building intelligent agents using the Microsoft Agent Framework with GitHub Models integration. You'll learn proven patterns and architectural approaches that make agents more robust, maintainable, and effective.

**Core Design Patterns Covered:**
- 🏗️ **Agent Factory Pattern**: Standardized agent creation and configuration
- 🔧 **Tool Registry Pattern**: Organized approach to managing agent capabilities
- 🧵 **Conversation Management**: Effective patterns for multi-turn interactions
- 🔄 **Response Processing**: Best practices for handling agent outputs

## 🎯 Key Architectural Concepts

### Design Principles
- **Separation of Concerns**: Clear boundaries between agent logic, tools, and configuration
- **Composability**: Building complex agents from reusable components
- **Extensibility**: Patterns that allow easy addition of new capabilities
- **Testability**: Design for easy unit testing and validation

### GitHub Models Integration
- **API Compatibility**: Leveraging OpenAI-compatible endpoints
- **Model Selection**: Choosing appropriate models for different use cases
- **Rate Limiting**: Handling API constraints gracefully
- **Error Recovery**: Robust error handling and retry patterns

## 🔧 Technical Architecture

### Core Components
- **Microsoft Agent Framework**: Python implementation with GitHub Models support
- **GitHub Models API**: Access to state-of-the-art language models
- **OpenAI Client Pattern**: Standardized API interaction patterns
- **Environment Configuration**: Secure and flexible configuration management

### Design Pattern Benefits
- **Maintainability**: Clear code organization and structure
- **Scalability**: Patterns that grow with your application needs
- **Reliability**: Proven approaches that handle edge cases
- **Performance**: Efficient resource utilization and API usage

## ⚙️ Prerequisites & Setup

**Required Dependencies:**
```bash

pip install agent-framework-core  -U

```

**Environment Configuration (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

**GitHub Models Access:**
- GitHub account with Models access
- Personal access token with appropriate permissions
- Understanding of rate limits and usage patterns

## 📚 Design Pattern Categories

### 1. **Creational Patterns**
- Agent factory and builder patterns
- Configuration management patterns
- Dependency injection for agent services

### 2. **Behavioral Patterns**
- Tool execution and orchestration
- Conversation flow management  
- Response processing and formatting

### 3. **Integration Patterns**
- GitHub Models API integration
- Error handling and retry logic
- Resource management and cleanup

## 🚀 Best Practices Demonstrated

- **Clean Architecture**: Layered design with clear responsibilities
- **Error Handling**: Comprehensive exception management
- **Configuration**: Environment-based setup for different environments
- **Testing**: Patterns that enable effective unit and integration testing
- **Documentation**: Self-documenting code with clear intent

Ready to explore professional agent design patterns? Let's build something robust! 🌟

In [19]:

import os                     
from random import randint    

from dotenv import load_dotenv  
from agent_framework import ChatAgent
from azure.identity import AzureCliCredential, DefaultAzureCredential
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework.openai import OpenAIChatClient

from azure.ai.projects.aio import AIProjectClient
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient

load_dotenv()

True

In [10]:
# 🛠️ Tool Function Design Pattern
def get_random_destination() -> str:
    """Get a random vacation destination using Repository Pattern.
    
    Returns:
        str: A randomly selected destination following consistent format
    """
    destinations = [
        "Barcelona, Spain",      
        "Paris, France",         
        "Berlin, Germany",       
        "Tokyo, Japan",          
        "Sydney, Australia",     
        "New York, USA",         
        "Cairo, Egypt",          
        "Cape Town, South Africa", 
        "Rio de Janeiro, Brazil",  
        "Bali, Indonesia"          
    ]
    
    return destinations[randint(0, len(destinations) - 1)]

In [11]:
AGENT_NAME ="TravelAgent"

AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

In [21]:
# Option 1: Using GitHub Models via OpenAIChatClient

chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id=os.environ.get("GITHUB_MODEL_ID"))

chat_agent = ChatAgent(
        name = AGENT_NAME,
        chat_client=chat_client,
        instructions=AGENT_INSTRUCTIONS,
        tools=[get_random_destination]
)

In [22]:
#Option 2: Using Azure OpenAi client to generate a local agent

chat_agent = AzureOpenAIChatClient(credential=DefaultAzureCredential()).create_agent(
    instructions=AGENT_INSTRUCTIONS,
    name = AGENT_NAME,
    tools=[get_random_destination]
)

In [ ]:
#Option 3: Using persisted Azure AI Foundry Agents 

chat_agent : ChatAgent
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],credential=credential)

try:
    # Create an agent that will persist
    created_agent = await project_client.agents.create_agent(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS
    )
    
    chat_agent = ChatAgent(
        chat_client=AzureAIAgentClient(project_client=project_client, agent_id=created_agent.id),
        tools=[get_random_destination])
    
    print(f"Agent {created_agent.name} created, and chat agent ready")
    
finally:
    print("ok")

In [ ]:
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id=os.environ.get("GITHUB_MODEL_ID"))

In [25]:
# Creating a new conversation instance with the agent storing the history of messages for this specific run
thread = chat_agent.get_new_thread()

response1 = await chat_agent.run("Plan me a day trip",thread= thread)

last_message = response1.messages[-1]
text_content = last_message.contents[0].text
print("Travel plan:")
print(text_content)

Travel plan:
Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:  
1. Plan a day trip to a specific location  
2. Suggest a random vacation destination  
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)  
4. Plan an alternative trip if you don't like my first suggestion  

Where would you like your day trip to take place? If you have a specific destination in mind, let me know and I’ll focus on that location! If not, I can suggest a random or themed day trip for you.


In [26]:
# Adding a new user message to change the plan, on top of the past first round of conversation
# Getting the answers as a stream : 

async for chunk in chat_agent.run_stream("I don't like that destination. Plan me another vacation.", thread=thread):
    if chunk.text:
        print(chunk.text, end="", flush=True)  # 📝 Display response as it generates


It seems you don’t have a destination in mind. Let me find another vacation idea for you! I’ll suggest a random destination. Please hold on for a moment.How about a trip to Paris, France? Known as "The City of Light," Paris is filled with romance, art, culture, and incredible food. Would you like me to plan a day trip or a longer vacation in Paris for you? Let me know your preferences!